# Heart Disease Prediction

## 1. Introduction

**Research Question:**  *How accurately can we predict the likelihood of heart disease based on the factors ___?*

The phrase "heart disease" can be referring to various different conditions relating to the heart. Mild symptoms can include chest pain and shortness of breath and extreme symptoms can include heart attacks and failures. A wide range of health attributes can affect the likelihood of a person having heart disease. It would be useful for doctors to be able to use these factors to predict if an individual has heart disease.

This project will examine the Heart Disease Data Set from the UCI Machine Learning repository. While there are datasets provided by four different hospitals, the page description says that the Cleveland database is the only one used by ML researchers. The Cleveland dataset is the one that we will analyze. This dataset is known to be a reliable source because cardiologist Dr. Robert Detrano conducted the study with V.A. Medical Center, Long Beach and Cleveland Clinic Foundation. The goal of this project is to determine the accuracy of predicting heart disease based on certain factors. The dataset provides numerous factors such as age, maximum heart rate, etc. (will be displayed below). 

## 2. Preliminary Data Analysis

### Loading Libraries

In [1]:
#load packages 
library(tidyverse)
library(repr)
library(tidymodels)

#set seed
set.seed(1234)

#number rows for table
options(repr.matrix.max.rows = 8)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

### Importing Data

In [2]:
link <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
untidy_cleveland <- read_csv(link, col_names = FALSE)
untidy_cleveland

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


Figure 1. Untidy tibble of Cleveland heart disease data

### Data Cleaning 

For this data to be tidy, column names must be changed, rows with "?" (NA) values must be removed, some columns must be changed to double data type, and the rightmost column must be changed to factor data type. 

In [3]:
#renaming the column names
colnames(untidy_cleveland) = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

tidy_cleveland = untidy_cleveland |> 
    filter(ca != "?", thal != "?") |>   #in ca and thal there were "?" values
    mutate(ca = as.integer(ca)) |>  #can change to <int> because no "?" and no decimals
    mutate(num = as.factor(num), sex = as.factor(sex), cp = as.factor(cp), fbs = as.factor(fbs), restecg = as.factor(restecg), exang = as.factor(exang), slope = as.factor(slope), thal = as.factor(thal))   #categorical data not <dbl>

tidy_cleveland
    

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<fct>,<int>,<fct>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
45,1,1,110,264,0,0,132,0,1.2,2,0,7.0,1
68,1,4,144,193,1,0,141,0,3.4,2,2,7.0,2
57,1,4,130,131,0,0,115,1,1.2,2,1,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1,3.0,1


Figure 2. Tidy tibble of Cleveland heart disease data

#### Attribute Information

The column names are given by the data set description on the UCI Machine Learning Repository page

**age:** age in years

**sex:** sex (1 = male, 0 = female)

**cp:** chest pain type (1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic)

**trestbps:** resting blood pressure (in mmHg)

**chol:** serum cholestoral (in mg/dl)

**fbs:** fasting blood sugar > 120 mg/dl (1 = true,  0 = false)

**restecg:** resting electrocardiographic results (0 = normal, 1 = stt abnormality, 2 = lv hypertrophy)

**thalach:** maximum heart rate achieved

**excang:** exercise induced angina (1 = yes, 0 = no)

**oldpeak:** ST depression induced by exercise relative to rest

**slope:** slope of the peak exercise ST segment (1 = upsloping, 2 = flat, 3 = downsloping)

**ca:** number of major vessels (0-3) colored by flourosopy

**thal:** 3 = normal, 6 = fixed defect, 7 = reversable defect

**num:** diagnosis of heart disease (1 >= heart disease exists, 0 = no risk of heart disease)

### Regrouping the response variable

Since num appears to be binary (no heart disease or heart disease) we can use mutate to change the values of num (this was searched how to do).

In [4]:
new_cleveland <- tidy_cleveland |> 
    mutate(disease = ifelse(num == 1 | num == 2 | num == 3 | num == 4, "Y", "N")) |> #if num >= 1, change to "Y" else "N" ("Y" = disease present, "N" = no disease)
    mutate(disease = as.factor(disease)) |>
    select(-num)

### Splitting into Training and Testing Sets

The data will be split into a training set and testing set and only the training set will be used to construct the classifier. After the classifier is built, its accuracy can be determined when it predicts the labels for the testing set (without disease values). The proportion of the data grouped as the training set will be 0.75 will the testing set is 0.25 of the data. These proportions are often what is seen in the textbook.

In [5]:
#stratifying disease such that sufficient proportions of 1 and 0 in each set
cleveland_split <- initial_split(new_cleveland, prop = 0.75, strata = disease) 

disease_training <- training(cleveland_split)

disease_testing <- testing(cleveland_split)

disease_training #tibble is 222 x 14, 297 * 0.75 = 222.75 so successfully split

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,disease
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<fct>,<int>,<fct>,<fct>
37,1,3,130,250,0,0,187,0,3.5,3,0,3.0,N
41,0,2,130,204,0,2,172,0,1.4,1,0,3.0,N
56,1,2,120,236,0,0,178,0,0.8,1,0,3.0,N
57,1,4,140,192,0,0,148,0,0.4,2,0,6.0,N
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,0,4,140,241,0,0,123,1,0.2,2,0,7.0,Y
45,1,1,110,264,0,0,132,0,1.2,2,0,7.0,Y
68,1,4,144,193,1,0,141,0,3.4,2,2,7.0,Y
57,0,2,130,236,0,2,174,0,0.0,2,1,3.0,Y


Figure 3. Training set with 75% of the total data

### Data Analysis

## 3. Methods

## 4. Expected outcomes and significance

## 5. Sources

Detrano, R. (1990). *Cleveland* [Data set]. V.A. Medical Center, Long Beach and Cleveland Clinic Foundation.      
     https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data